# 🔬 Comprehensive OpenSearch Analysis: Analyzers, BM25, Neural & Hybrid Search

This notebook performs a comprehensive comparison of different text analysis and search strategies using the SQUAD dataset:

## 📊 Workflow Overview

```mermaid
%%{init: {'theme':'base', 'themeVariables': { 'primaryColor':'#4A90E2','primaryTextColor':'#fff','primaryBorderColor':'#357ABD','lineColor':'#F28C28','secondaryColor':'#50E3C2','tertiaryColor':'#F5A623','background':'#ffffff','mainBkg':'#4A90E2','secondBkg':'#50E3C2','tertiaryBkg':'#F5A623'}}}%%

graph TB
    Start([📦 SQUAD Dataset<br/>~87k documents]) --> Load[🔄 Data Preprocessing]
    
    Load --> Phase1{Phase 1:<br/>Text Analyzers}
    
    Phase1 --> A1[📝 Standard Analyzer<br/>General purpose]
    Phase1 --> A2[🇬🇧 English Analyzer<br/>Stemming + stopwords]
    Phase1 --> A3[🔑 Keyword Analyzer<br/>Exact matching]
    Phase1 --> A4[⚡ Whitespace Analyzer<br/>Space splitting]
    Phase1 --> A5[🔤 Simple Analyzer<br/>Lowercase only]
    
    A1 --> Ingest1[⏱️ Bulk Ingest + Metrics]
    A2 --> Ingest1
    A3 --> Ingest1
    A4 --> Ingest1
    A5 --> Ingest1
    
    Ingest1 --> Compare1[📊 Table 1:<br/>Ingestion Performance<br/>Time, Throughput, Size]
    
    Compare1 --> Query1[🔍 BM25 Queries<br/>match, match_phrase, multi_match]
    
    Query1 --> Compare2[📊 Table 2:<br/>Query Performance<br/>Latency, Relevance, Results]
    
    Compare2 --> Phase2{Phase 2:<br/>Neural Search}
    
    Phase2 --> Model[🤖 Deploy Model<br/>msmarco-distilbert-base-tas-b<br/>HuggingFace]
    
    Model --> Embed[🧬 Generate Vectors<br/>title, context, question, answers]
    
    Embed --> IngestVec[📥 Ingest with k-NN<br/>Vector embeddings]
    
    IngestVec --> Phase3{Phase 3:<br/>Search Comparison}
    
    Phase3 --> BM25[🎯 BM25 Search<br/>Lexical matching]
    Phase3 --> Neural[🧠 Neural Search<br/>k-NN vector similarity]
    Phase3 --> Hybrid[⚡ Hybrid Search<br/>BM25 + Neural combined]
    
    BM25 --> Final[📊 Table 3:<br/>Final Comparison<br/>BM25 vs Neural vs Hybrid<br/>Latency, Precision, Use Cases]
    Neural --> Final
    Hybrid --> Final
    
    Final --> End([✅ Complete Analysis<br/>with Recommendations])
    
    style Start fill:#4A90E2,stroke:#357ABD,stroke-width:3px,color:#fff
    style Phase1 fill:#F5A623,stroke:#D68910,stroke-width:3px,color:#fff
    style Phase2 fill:#F5A623,stroke:#D68910,stroke-width:3px,color:#fff
    style Phase3 fill:#F5A623,stroke:#D68910,stroke-width:3px,color:#fff
    style Compare1 fill:#50E3C2,stroke:#2ECC71,stroke-width:2px,color:#000
    style Compare2 fill:#50E3C2,stroke:#2ECC71,stroke-width:2px,color:#000
    style Final fill:#50E3C2,stroke:#2ECC71,stroke-width:2px,color:#000
    style Model fill:#E74C3C,stroke:#C0392B,stroke-width:2px,color:#fff
    style End fill:#2ECC71,stroke:#27AE60,stroke-width:3px,color:#fff
    style BM25 fill:#9B59B6,stroke:#8E44AD,stroke-width:2px,color:#fff
    style Neural fill:#9B59B6,stroke:#8E44AD,stroke-width:2px,color:#fff
    style Hybrid fill:#9B59B6,stroke:#8E44AD,stroke-width:2px,color:#fff
```

## 🎯 Analysis Goals

1. **Analyzer Comparison**: Test multiple text analyzers and measure ingestion performance
2. **Query Strategy**: Compare search effectiveness across different analyzers
3. **Neural Search**: Implement vector-based semantic search using sentence transformers
4. **Hybrid Approach**: Combine lexical (BM25) and semantic (Neural) search for optimal results

---

## Config

In [1]:
import sys
import os
%load_ext autoreload
%autoreload 2

# Get the current working directory of the notebook
current_dir = os.getcwd()

DATA_DIR = os.path.abspath(os.path.join(current_dir, '../../0. DATA'))

# Construct the path to the directory levels up
module_paths = [os.path.abspath(os.path.join(current_dir, '../../')),] 

# Add the module path to sys.path if it's not already there
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

try:
    import helpers as hp
except ImportError as e:
    raise ImportError(f"Error importing modules: {e}")

In [2]:
from opensearchpy import OpenSearch
from opensearch_py_ml.ml_commons import MLCommonClient
import time
import pandas as pd
from opensearchpy import OpenSearch, helpers

IS_AUTH = True # Set to False if security is disabled
HOST = 'localhost'  # Replace with your OpenSearch host, if running everything locally use 'localhost'

if IS_AUTH:
    # Initialize the OpenSearch client
    os_client = OpenSearch(
        hosts=[{'host': HOST, 'port': 9200}],
        http_auth=('admin', 'Developer@123'),  # Replace with your credentials
        use_ssl=True,
        verify_certs=False,
        ssl_show_warn=False
    )
else:
    # Initialize the OpenSearch client without authentication
    os_client = OpenSearch(
        hosts=[{'host': HOST, 'port': 9200}],
        use_ssl=False,
        verify_certs=False,
        ssl_assert_hostname = False,
        ssl_show_warn=False
    )

# Initialize ML Commons client
ml_client = MLCommonClient(os_client)

# Check if cluster is up
if (os_client.ping()):
    print("Connected to OpenSearch cluster")

Connected to OpenSearch cluster


## Read SQUAD Dataset

In [3]:
from IPython.display import display, HTML
import pandas as pd
import json
df_squad_train = pd.read_parquet(f"{DATA_DIR}/SQUAD-train.parquet")

# Check id is unique i.e. count of rows is same as count of unique ids
if {len(df_squad_train)} == {df_squad_train['id'].nunique()}:
    print("id is unique i.e. primary key")
else:
    print("id is not unique")

# Print pandas memory usage in MB
memory_usage = df_squad_train.memory_usage(deep=True)
memory_usage_mb = memory_usage / (1024 * 1024)
display(memory_usage_mb)
print(f"\nTotal memory usage: {memory_usage_mb.sum():.2f} MB")

# Enable word wrap for better readability in Jupyter Notebook
display(HTML("<style>.output_area pre {white-space: pre-wrap; word-break: break-word;}</style>")) 
# Display the first few rows of the dataframe
print("First few rows of the SQuAD training dataset:")
display(df_squad_train.head())

# Print one row as dictionary pretty print
print("One row as dictionary:")
import numpy as np

def convert_to_serializable(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    else:
        return obj

row_dict = df_squad_train.iloc[0].to_dict()
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)
print(json.dumps(convert_to_serializable(row_dict), indent=4, ensure_ascii=False))


id is unique i.e. primary key


Index        0.000126
id           6.098487
title        5.291620
context     83.031409
question     9.089622
answers     16.039856
dtype: float64


Total memory usage: 119.55 MB


First few rows of the SQuAD training dataset:


,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


One row as dictionary:
{
    "id": "5733be284776f41900661182",
    "title": "University_of_Notre_Dame",
    "context": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",
    "question": "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?",
    "answers": {
        "text": [
            "Saint Bernadette Soubirous"
        ],
  

# 📊 Phase 1: Text Analyzer Comparison

In this phase, we'll create multiple indices with different text analyzers and compare their ingestion performance.

In [36]:
# Define different analyzer configurations
analyzer_configs = {
    "standard": {
        "description": "Default analyzer with grammar based tokenization and lowercase filter",
        "settings": {
            "analysis": {
                "analyzer": {
                    "default": {
                        "type": "standard"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "title": {"type": "text", "analyzer": "standard"},
                "context": {"type": "text", "analyzer": "standard"},
                "question": {"type": "text", "analyzer": "standard"},
                "answers": {
                    "type": "nested",
                    "properties": {
                        "text": {"type": "text", "analyzer": "standard"},
                        "answer_start": {"type": "integer"}
                    }
                }
            }
        }
    },
    "english": {
        "description": "English analyzer with stemming and stopwords removal",
        "settings": {
            "analysis": {
                "analyzer": {
                    "default": {
                        "type": "english"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "title": {"type": "text", "analyzer": "english"},
                "context": {"type": "text", "analyzer": "english"},
                "question": {"type": "text", "analyzer": "english"},
                "answers": {
                    "type": "nested",
                    "properties": {
                        "text": {"type": "text", "analyzer": "english"},
                        "answer_start": {"type": "integer"}
                    }
                }
            }
        }
    },
    "keyword": {
        "description": "Keyword analyzer for exact matching (no tokenization)",
        "settings": {
            "analysis": {
                "analyzer": {
                    "default": {
                        "type": "keyword"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "title": {"type": "keyword"},
                "context": {"type": "text"},  # Keep text for context as it's too long
                "question": {"type": "text"},  # Keep text for question
                "answers": {
                    "type": "nested",
                    "properties": {
                        "text": {"type": "text"},
                        "answer_start": {"type": "integer"}
                    }
                }
            }
        }
    },
    "whitespace": {
        "description": "Whitespace analyzer splits on whitespace only",
        "settings": {
            "analysis": {
                "analyzer": {
                    "default": {
                        "type": "whitespace"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "title": {"type": "text", "analyzer": "whitespace"},
                "context": {"type": "text", "analyzer": "whitespace"},
                "question": {"type": "text", "analyzer": "whitespace"},
                "answers": {
                    "type": "nested",
                    "properties": {
                        "text": {"type": "text", "analyzer": "whitespace"},
                        "answer_start": {"type": "integer"}
                    }
                }
            }
        }
    },
    "simple": {
        "description": "Simple analyzer divides text at non-letters and lowercase",
        "settings": {
            "analysis": {
                "analyzer": {
                    "default": {
                        "type": "simple"
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "title": {"type": "text", "analyzer": "simple"},
                "context": {"type": "text", "analyzer": "simple"},
                "question": {"type": "text", "analyzer": "simple"},
                "answers": {
                    "type": "nested",
                    "properties": {
                        "text": {"type": "text", "analyzer": "simple"},
                        "answer_start": {"type": "integer"}
                    }
                }
            }
        }
    }
}

print("📝 Analyzer Configurations:")
print("=" * 80)
for name, config in analyzer_configs.items():
    print(f"\n🔹 {name.upper()}: {config['description']}")
print("\n" + "=" * 80)

📝 Analyzer Configurations:

🔹 STANDARD: Default analyzer with grammar based tokenization and lowercase filter

🔹 ENGLISH: English analyzer with stemming and stopwords removal

🔹 KEYWORD: Keyword analyzer for exact matching (no tokenization)

🔹 WHITESPACE: Whitespace analyzer splits on whitespace only

🔹 SIMPLE: Simple analyzer divides text at non-letters and lowercase



In [39]:
# Function to prepare documents for bulk ingestion
def prepare_bulk_data(df, index_name):
    """
    Prepare data for bulk ingestion
    """
    for _, row in df.iterrows():
        doc = {
            "_index": index_name,
            "_id": row['id'],
            "_source": {
                "id": row['id'],
                "title": row['title'],
                "context": row['context'],
                "question": row['question'],
                "answers": row['answers']
            }
        }
        yield doc

# Function to perform bulk ingestion with metrics
def bulk_ingest_with_metrics(os_client, df, index_name, analyzer_name):
    """
    Perform bulk ingestion and measure performance metrics
    """
    print(f"\n🚀 Starting bulk ingestion for '{analyzer_name}' analyzer...")
    print(f"   Index: {index_name}")
    print(f"   Documents: {len(df):,}")
    
    start_time = time.time()
    
    try:
        # Perform bulk ingestion with progress tracking
        success_count = 0
        error_count = 0
        
        # Use helpers.bulk for efficient ingestion
        # Using streaming API with stats_only=False to get detailed responses
        result_gen = helpers.bulk(
            os_client, 
            prepare_bulk_data(df, index_name),
            chunk_size=500,
            request_timeout=60,
            raise_on_error=False,
            stats_only=False
        )
        
        for item in result_gen:
            # Each item is a tuple: (success_boolean, response_dict)
            if isinstance(item, tuple) and len(item) == 2:
                ok, response = item
                if ok:
                    success_count += 1
                else:
                    error_count += 1
            else:
                # Fallback if response format is unexpected
                success_count += 1
        
        elapsed_time = time.time() - start_time
        
        # Refresh index to make documents searchable
        os_client.indices.refresh(index=index_name)
        
        # Get index stats
        stats = os_client.indices.stats(index=index_name)
        index_size_bytes = stats['indices'][index_name]['total']['store']['size_in_bytes']
        index_size_mb = index_size_bytes / (1024 * 1024)
        doc_count = stats['indices'][index_name]['total']['docs']['count']
        
        docs_per_second = len(df) / elapsed_time if elapsed_time > 0 else 0
        
        return {
            "analyzer": analyzer_name,
            "index": index_name,
            "total_docs": len(df),
            "success": success_count,
            "errors": error_count,
            "time_seconds": round(elapsed_time, 2),
            "docs_per_second": round(docs_per_second, 2),
            "index_size_mb": round(index_size_mb, 2),
            "indexed_docs": doc_count,
            "status": "✅ Success" if error_count == 0 else f"⚠️ {error_count} errors"
        }
    
    except Exception as e:
        elapsed_time = time.time() - start_time
        return {
            "analyzer": analyzer_name,
            "index": index_name,
            "total_docs": len(df),
            "success": 0,
            "errors": len(df),
            "time_seconds": round(elapsed_time, 2),
            "docs_per_second": 0,
            "index_size_mb": 0,
            "indexed_docs": 0,
            "status": f"❌ Failed: {str(e)[:50]}"
        }

print("✅ Helper functions defined")

✅ Helper functions defined


In [40]:
# Create indices and perform bulk ingestion for each analyzer
ingestion_results = []

for analyzer_name, config in analyzer_configs.items():
    index_name = f"squad_{analyzer_name}_analyzer"
    
    # Delete index if exists
    if os_client.indices.exists(index=index_name):
        os_client.indices.delete(index=index_name)
        print(f"🗑️  Deleted existing index: {index_name}")
    
    # Create index with specific analyzer settings
    os_client.indices.create(
        index=index_name,
        body={
            "settings": config["settings"],
            "mappings": config["mappings"]
        }
    )
    print(f"✨ Created index: {index_name}")
    
    # Perform bulk ingestion with metrics
    result = bulk_ingest_with_metrics(os_client, df_squad_train, index_name, analyzer_name)
    ingestion_results.append(result)
    
    print(f"   ⏱️  Time: {result['time_seconds']}s | 📊 Throughput: {result['docs_per_second']} docs/s")
    print(f"   💾 Index Size: {result['index_size_mb']} MB | {result['status']}")

print("\n" + "=" * 80)
print("✅ All indices created and data ingested!")
print("=" * 80)

🗑️  Deleted existing index: squad_standard_analyzer
✨ Created index: squad_standard_analyzer

🚀 Starting bulk ingestion for 'standard' analyzer...
   Index: squad_standard_analyzer
   Documents: 87,599
   ⏱️  Time: 9.54s | 📊 Throughput: 9178.4 docs/s
   💾 Index Size: 104.04 MB | ✅ Success
🗑️  Deleted existing index: squad_english_analyzer
   ⏱️  Time: 9.54s | 📊 Throughput: 9178.4 docs/s
   💾 Index Size: 104.04 MB | ✅ Success
🗑️  Deleted existing index: squad_english_analyzer
✨ Created index: squad_english_analyzer

🚀 Starting bulk ingestion for 'english' analyzer...
   Index: squad_english_analyzer
   Documents: 87,599
✨ Created index: squad_english_analyzer

🚀 Starting bulk ingestion for 'english' analyzer...
   Index: squad_english_analyzer
   Documents: 87,599
   ⏱️  Time: 10.34s | 📊 Throughput: 8470.54 docs/s
   💾 Index Size: 98.38 MB | ✅ Success
🗑️  Deleted existing index: squad_keyword_analyzer
   ⏱️  Time: 10.34s | 📊 Throughput: 8470.54 docs/s
   💾 Index Size: 98.38 MB | ✅ Succe

## 📊 Table 1: Ingestion Performance Comparison

In [41]:
# Create comprehensive comparison table for ingestion
df_ingestion = pd.DataFrame(ingestion_results)

# Sort by throughput (docs per second) descending
df_ingestion_sorted = df_ingestion.sort_values('docs_per_second', ascending=False)

# Create a styled display
from IPython.display import display, HTML

# Add ranking
df_ingestion_sorted['rank'] = range(1, len(df_ingestion_sorted) + 1)

# Reorder columns for better presentation
df_display = df_ingestion_sorted[[
    'rank', 'analyzer', 'total_docs', 'success', 'errors',
    'time_seconds', 'docs_per_second', 'index_size_mb', 'status'
]].copy()

# Rename columns for better readability
df_display.columns = [
    '🏆 Rank', '🔧 Analyzer', '📄 Total Docs', '✅ Success', '❌ Errors',
    '⏱️ Time (s)', '📊 Throughput (docs/s)', '💾 Size (MB)', '📌 Status'
]

# Create HTML table with styling
html_table = df_display.to_html(index=False, escape=False, border=0)

# Add custom CSS styling
styled_html = f"""
<style>
    .ingestion-table {{
        width: 100%;
        border-collapse: collapse;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        margin: 20px 0;
    }}
    .ingestion-table th {{
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 12px;
        text-align: left;
        font-weight: 600;
        font-size: 14px;
    }}
    .ingestion-table td {{
        padding: 10px 12px;
        border-bottom: 1px solid #e0e0e0;
        font-size: 13px;
    }}
    .ingestion-table tr:hover {{
        background-color: #f5f5f5;
    }}
    .ingestion-table tr:nth-child(even) {{
        background-color: #fafafa;
    }}
</style>
<div class="ingestion-table">
    {html_table.replace('<table', '<table class="ingestion-table"')}
</div>
"""

display(HTML(styled_html))

# Print summary statistics
print("\n" + "=" * 80)
print("📈 INGESTION PERFORMANCE SUMMARY")
print("=" * 80)
print(f"🏆 Fastest: {df_ingestion_sorted.iloc[0]['analyzer'].upper()} "
      f"({df_ingestion_sorted.iloc[0]['docs_per_second']} docs/s)")
print(f"🐌 Slowest: {df_ingestion_sorted.iloc[-1]['analyzer'].upper()} "
      f"({df_ingestion_sorted.iloc[-1]['docs_per_second']} docs/s)")
print(f"📊 Average Throughput: {df_ingestion_sorted['docs_per_second'].mean():.2f} docs/s")
print(f"💾 Average Index Size: {df_ingestion_sorted['index_size_mb'].mean():.2f} MB")
print(f"⏱️ Total Ingestion Time: {df_ingestion_sorted['time_seconds'].sum():.2f}s")
print("=" * 80)

🏆 Rank,🔧 Analyzer,📄 Total Docs,✅ Success,❌ Errors,⏱️ Time (s),📊 Throughput (docs/s),💾 Size (MB),📌 Status
1,keyword,87599,2,0,6.51,13451.89,84.60,✅ Success
2,simple,87599,2,0,9.03,9698.72,36.58,✅ Success
3,whitespace,87599,2,0,9.09,9639.35,109.64,✅ Success
4,standard,87599,2,0,9.54,9178.40,104.04,✅ Success
5,english,87599,2,0,10.34,8470.54,98.38,✅ Success



📈 INGESTION PERFORMANCE SUMMARY
🏆 Fastest: KEYWORD (13451.89 docs/s)
🐌 Slowest: ENGLISH (8470.54 docs/s)
📊 Average Throughput: 10087.78 docs/s
💾 Average Index Size: 86.65 MB
⏱️ Total Ingestion Time: 44.51s


# 🔍 Phase 2: Query Performance Comparison

Now let's test how different analyzers perform for searching.

In [42]:
# Define sample search queries
sample_queries = [
    {
        "name": "Simple Match Query",
        "type": "match",
        "query_text": "What is the capital of France?",
        "field": "question"
    },
    {
        "name": "Multi-field Search",
        "type": "multi_match",
        "query_text": "machine learning algorithms",
        "fields": ["title", "context", "question"]
    },
    {
        "name": "Phrase Match",
        "type": "match_phrase",
        "query_text": "neural network",
        "field": "context"
    },
    {
        "name": "Context Search",
        "type": "match",
        "query_text": "Super Bowl",
        "field": "context"
    }
]

# Function to execute query and measure performance
def execute_search_with_metrics(os_client, index_name, query_config):
    """
    Execute a search query and measure performance
    """
    # Build query based on type
    if query_config["type"] == "match":
        query_body = {
            "query": {
                "match": {
                    query_config["field"]: query_config["query_text"]
                }
            },
            "size": 10
        }
    elif query_config["type"] == "multi_match":
        query_body = {
            "query": {
                "multi_match": {
                    "query": query_config["query_text"],
                    "fields": query_config["fields"]
                }
            },
            "size": 10
        }
    elif query_config["type"] == "match_phrase":
        query_body = {
            "query": {
                "match_phrase": {
                    query_config["field"]: query_config["query_text"]
                }
            },
            "size": 10
        }
    
    start_time = time.time()
    
    try:
        response = os_client.search(index=index_name, body=query_body)
        elapsed_time = (time.time() - start_time) * 1000  # Convert to milliseconds
        
        hits = response['hits']['hits']
        total_hits = response['hits']['total']['value']
        max_score = response['hits']['max_score'] if hits else 0
        
        return {
            "latency_ms": round(elapsed_time, 2),
            "total_hits": total_hits,
            "returned_hits": len(hits),
            "max_score": round(max_score, 4) if max_score else 0,
            "status": "✅ Success"
        }
    
    except Exception as e:
        elapsed_time = (time.time() - start_time) * 1000
        return {
            "latency_ms": round(elapsed_time, 2),
            "total_hits": 0,
            "returned_hits": 0,
            "max_score": 0,
            "status": f"❌ Error: {str(e)[:30]}"
        }

print("✅ Query execution functions defined")
print("\n📝 Sample Queries:")
for i, q in enumerate(sample_queries, 1):
    print(f"  {i}. {q['name']}: '{q['query_text']}'  [{q['type']}]")

✅ Query execution functions defined

📝 Sample Queries:
  1. Simple Match Query: 'What is the capital of France?'  [match]
  2. Multi-field Search: 'machine learning algorithms'  [multi_match]
  3. Phrase Match: 'neural network'  [match_phrase]
  4. Context Search: 'Super Bowl'  [match]


In [43]:
# Execute queries across all analyzers
query_results = []

for query_config in sample_queries:
    print(f"\n{'='*80}")
    print(f"🔍 Executing: {query_config['name']}")
    print(f"   Query: '{query_config['query_text']}'")
    print(f"{'='*80}")
    
    for analyzer_name in analyzer_configs.keys():
        index_name = f"squad_{analyzer_name}_analyzer"
        
        result = execute_search_with_metrics(os_client, index_name, query_config)
        
        query_results.append({
            "query_name": query_config['name'],
            "query_text": query_config['query_text'],
            "query_type": query_config['type'],
            "analyzer": analyzer_name,
            "index": index_name,
            **result
        })
        
        print(f"  📊 {analyzer_name:12s} | "
              f"⏱️ {result['latency_ms']:6.2f}ms | "
              f"📄 {result['total_hits']:5d} hits | "
              f"⭐ {result['max_score']:.4f} score")

print(f"\n{'='*80}")
print("✅ All queries executed!")
print(f"{'='*80}")


🔍 Executing: Simple Match Query
   Query: 'What is the capital of France?'
  📊 standard     | ⏱️  59.06ms | 📄 10000 hits | ⭐ 6.2091 score
  📊 english      | ⏱️  18.39ms | 📄 10000 hits | ⭐ 5.7593 score
  📊 keyword      | ⏱️   3.64ms | 📄     0 hits | ⭐ 0.0000 score
  📊 whitespace   | ⏱️  13.09ms | 📄 10000 hits | ⭐ 5.6961 score
  📊 simple       | ⏱️   6.88ms | 📄 10000 hits | ⭐ 6.1711 score

🔍 Executing: Multi-field Search
   Query: 'machine learning algorithms'
  📊 standard     | ⏱️  20.66ms | 📄  1489 hits | ⭐ 7.5463 score
  📊 english      | ⏱️   7.47ms | 📄  2504 hits | ⭐ 7.5365 score
  📊 keyword      | ⏱️   5.66ms | 📄     0 hits | ⭐ 0.0000 score
  📊 whitespace   | ⏱️   5.52ms | 📄   955 hits | ⭐ 6.1530 score
  📊 simple       | ⏱️   5.85ms | 📄  1528 hits | ⭐ 7.5252 score

🔍 Executing: Phrase Match
   Query: 'neural network'
  📊 standard     | ⏱️  15.41ms | 📄     4 hits | ⭐ 4.5246 score
  📊 english      | ⏱️   3.92ms | 📄    14 hits | ⭐ 5.6011 score
  📊 keyword      | ⏱️   2.29ms | 📄     0 

## 📊 Table 2: Query Performance Comparison

In [44]:
# Create comprehensive query comparison table
df_queries = pd.DataFrame(query_results)

# Create pivot table for better visualization
df_query_pivot = df_queries.pivot_table(
    index=['query_name', 'query_text'],
    columns='analyzer',
    values=['latency_ms', 'total_hits', 'max_score'],
    aggfunc='first'
)

# Display detailed results grouped by query
print("="*120)
print("📊 QUERY PERFORMANCE BY ANALYZER")
print("="*120)

for query_name in df_queries['query_name'].unique():
    query_data = df_queries[df_queries['query_name'] == query_name]
    
    print(f"\n🔍 {query_name}")
    print(f"   Query: '{query_data.iloc[0]['query_text']}'")
    print(f"   Type: {query_data.iloc[0]['query_type']}")
    print("-" * 120)
    
    # Create display dataframe
    display_df = query_data[['analyzer', 'latency_ms', 'total_hits', 'max_score', 'status']].copy()
    display_df = display_df.sort_values('latency_ms')
    display_df['rank'] = range(1, len(display_df) + 1)
    
    display_df = display_df[['rank', 'analyzer', 'latency_ms', 'total_hits', 'max_score', 'status']]
    display_df.columns = ['🏆', 'Analyzer', 'Latency (ms)', 'Total Hits', 'Max Score', 'Status']
    
    print(display_df.to_string(index=False))
    
    # Print winner
    fastest = display_df.iloc[0]
    print(f"\n   🏆 Winner: {fastest['Analyzer'].upper()} - {fastest['Latency (ms)']}ms latency")
    print()

# Create summary statistics table
print("\n" + "="*120)
print("📈 SUMMARY STATISTICS BY ANALYZER")
print("="*120)

summary_stats = df_queries.groupby('analyzer').agg({
    'latency_ms': ['mean', 'min', 'max', 'std'],
    'total_hits': 'mean',
    'max_score': 'mean'
}).round(2)

summary_stats.columns = ['Avg Latency', 'Min Latency', 'Max Latency', 'Std Dev', 'Avg Hits', 'Avg Score']
summary_stats = summary_stats.sort_values('Avg Latency')
summary_stats['rank'] = range(1, len(summary_stats) + 1)
summary_stats = summary_stats[['rank', 'Avg Latency', 'Min Latency', 'Max Latency', 'Std Dev', 'Avg Hits', 'Avg Score']]

print(summary_stats.to_string())

# Create styled HTML table
html_summary = f"""
<style>
    .query-table {{
        width: 100%;
        border-collapse: collapse;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        box-shadow: 0 2px 8px rgba(0,0,0,0.1);
        margin: 20px 0;
    }}
    .query-table th {{
        background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
        color: white;
        padding: 12px;
        text-align: left;
        font-weight: 600;
        font-size: 14px;
    }}
    .query-table td {{
        padding: 10px 12px;
        border-bottom: 1px solid #e0e0e0;
        font-size: 13px;
    }}
    .query-table tr:hover {{
        background-color: #fff5f5;
    }}
    .query-table tr:nth-child(even) {{
        background-color: #fef5f5;
    }}
</style>
<div class="query-table">
    <h3>🏆 Query Performance Leaderboard</h3>
    {summary_stats.to_html(index=True, escape=False).replace('<table', '<table class="query-table"')}
</div>
"""

display(HTML(html_summary))

print("\n" + "="*120)
print(f"🏆 Overall Fastest Analyzer: {summary_stats.index[0].upper()} "
      f"(Avg: {summary_stats.iloc[0]['Avg Latency']:.2f}ms)")
print(f"⭐ Best Relevance Scores: {summary_stats['Avg Score'].idxmax().upper()} "
      f"(Avg Score: {summary_stats['Avg Score'].max():.4f})")
print("="*120)

📊 QUERY PERFORMANCE BY ANALYZER

🔍 Simple Match Query
   Query: 'What is the capital of France?'
   Type: match
------------------------------------------------------------------------------------------------------------------------
 🏆   Analyzer  Latency (ms)  Total Hits  Max Score    Status
 1    keyword          3.64           0     0.0000 ✅ Success
 2     simple          6.88       10000     6.1711 ✅ Success
 3 whitespace         13.09       10000     5.6961 ✅ Success
 4    english         18.39       10000     5.7593 ✅ Success
 5   standard         59.06       10000     6.2091 ✅ Success

   🏆 Winner: KEYWORD - 3.64ms latency


🔍 Multi-field Search
   Query: 'machine learning algorithms'
   Type: multi_match
------------------------------------------------------------------------------------------------------------------------
 🏆   Analyzer  Latency (ms)  Total Hits  Max Score    Status
 1 whitespace          5.52         955     6.1530 ✅ Success
 2    keyword          5.66        

,rank,Avg Latency,Min Latency,Max Latency,Std Dev,Avg Hits,Avg Score
analyzer,,,,,,,
keyword,1,3.82,2.29,5.66,1.39,0.00,0.00
simple,2,5.25,4.13,6.88,1.35,3043.50,6.71
whitespace,3,6.84,3.51,13.09,4.26,2850.00,6.39
english,4,8.93,3.92,18.39,6.48,3296.50,6.80
standard,5,25.44,6.63,59.06,23.15,3033.75,6.69



🏆 Overall Fastest Analyzer: KEYWORD (Avg: 3.82ms)
⭐ Best Relevance Scores: ENGLISH (Avg Score: 6.8000)


# 🧠 Phase 3: Neural Search with Sentence Transformers

Now we'll deploy a sentence transformer model and create vector embeddings for semantic search.

In [48]:
# Register and deploy the sentence transformer model
print("="*80)
print("🤖 Registering and deploying ML model...")
print("="*80)

try:
    model_response = ml_client.register_pretrained_model(
        model_name="huggingface/sentence-transformers/msmarco-distilbert-base-tas-b",
        model_version="1.0.1",
        model_format="TORCH_SCRIPT",
        deploy_model=True,
        wait_until_deployed=True
    )
    model_id = model_response
    print(f"✅ Model registered successfully!")
    print(f"   Model ID: {model_id}")
    
    # Wait for model to be fully deployed
    print("\n⏳ Waiting for model deployment...")
    max_wait_time = 300  # 5 minutes max wait
    start_time = time.time()
    
    while True:
        model_info = ml_client.get_model_info(model_id)
        model_state = model_info.get('model_state', 'UNKNOWN')
        print(f"   Current model state: {model_state}")
        
        if model_state == 'DEPLOYED':
            print("✅ Model deployed successfully!")
            break
        
        if time.time() - start_time > max_wait_time:
            print("⚠️  Warning: Model deployment timeout. Proceeding anyway...")
            break
        
        time.sleep(5)
    
    print(f"\n{'='*80}")
    print(f"✅ Model is ready for use")
    print(f"   Model ID: {model_id}")
    print(f"{'='*80}")

except Exception as e:
    print(f"\n❌ Error with model setup: {e}")
    print("⚠️  You may need to manually register/deploy the model")
    print("   Continuing with a placeholder model_id...")
    model_id = "PLACEHOLDER_MODEL_ID"

🤖 Registering and deploying ML model...
Model was registered successfully. Model Id:  nD4n_5kBHWwV4qjRGNg4
nD4n_5kBHWwV4qjRGNg4
Task ID: rtYn_5kBDLL71ClipzUm
Model was registered successfully. Model Id:  nD4n_5kBHWwV4qjRGNg4
nD4n_5kBHWwV4qjRGNg4
Task ID: rtYn_5kBDLL71ClipzUm
Model deployed successfully
✅ Model registered successfully!
   Model ID: nD4n_5kBHWwV4qjRGNg4

⏳ Waiting for model deployment...
   Current model state: DEPLOYED
✅ Model deployed successfully!

✅ Model is ready for use
   Model ID: nD4n_5kBHWwV4qjRGNg4
Model deployed successfully
✅ Model registered successfully!
   Model ID: nD4n_5kBHWwV4qjRGNg4

⏳ Waiting for model deployment...
   Current model state: DEPLOYED
✅ Model deployed successfully!

✅ Model is ready for use
   Model ID: nD4n_5kBHWwV4qjRGNg4


In [51]:
# Generate embeddings for a sample of data first (for performance)
# Using a smaller sample for demonstration - adjust as needed
SAMPLE_SIZE = 1000  # Adjust based on your needs
df_sample = df_squad_train.head(SAMPLE_SIZE).copy()

print(f"🧬 Generating vector embeddings for {len(df_sample):,} documents")
print(f"   Fields to vectorize: title, context, question, answers.text")
print("="*80)

# Verify model is ready before generating embeddings
print(f"\n🔍 Verifying model deployment...")
print(f"   Model ID: {model_id}")

if model_id == "PLACEHOLDER_MODEL_ID":
    print("❌ Model deployment failed in previous step!")
    print("   Please re-run the model registration cell (cell 19) and ensure it completes successfully.")
    print("   The model state should be 'DEPLOYED' before continuing.")
    raise Exception("Invalid model_id. Cannot generate embeddings without a deployed model.")

try:
    # Check model status
    model_info = ml_client.get_model_info(model_id)
    model_state = model_info.get('model_state', 'UNKNOWN')
    print(f"   Model state: {model_state}")
    
    if model_state != 'DEPLOYED':
        print(f"⚠️  Warning: Model state is '{model_state}', not 'DEPLOYED'")
        print("   Embedding generation may fail. Consider re-deploying the model.")
    else:
        print("✅ Model is deployed and ready!")
except Exception as e:
    print(f"⚠️  Warning: Could not verify model status: {e}")
    print("   Proceeding anyway, but embedding generation may fail.")

print("="*80)

# Function to generate embeddings using ML Commons
def generate_embedding(text, model_id):
    """Generate embedding for given text using deployed model"""
    if not text or text.strip() == "":
        return None
    
    # Check if model_id is valid
    if model_id == "PLACEHOLDER_MODEL_ID" or not model_id:
        print(f"   ⚠️  Invalid model_id: {model_id}. Model deployment may have failed.")
        return None
    
    try:
        # Truncate text if too long (model has token limits)
        max_chars = 500
        text = text[:max_chars] if len(text) > max_chars else text
        
        # Use direct API call for more reliable embedding generation
        request_body = {
            "text_docs": [text],
            "return_number": True,
            "target_response": ["sentence_embedding"]
        }
        
        response = os_client.transport.perform_request(
            'POST',
            f'/_plugins/_ml/models/{model_id}/_predict',
            body=request_body
        )
        
        if 'inference_results' in response:
            embeddings = response['inference_results'][0]['output'][0]['data']
            return embeddings
        return None
    except Exception as e:
        print(f"   ⚠️  Embedding error: {str(e)[:150]}")
        return None

# Generate embeddings for all fields
print("\n🔄 Generating embeddings...")
embedding_start = time.time()

# Create lists to store embeddings
title_embeddings = []
context_embeddings = []
question_embeddings = []
answer_embeddings = []

successful_embeddings = 0
failed_embeddings = 0

for idx, row in df_sample.iterrows():
    if idx % 100 == 0:
        elapsed = time.time() - embedding_start
        rate = idx / elapsed if elapsed > 0 else 0
        print(f"   Progress: {idx}/{len(df_sample)} ({(idx/len(df_sample)*100):.1f}%) - {rate:.1f} docs/s")
    
    # Generate embeddings for each field
    title_emb = generate_embedding(str(row['title']), model_id)
    context_emb = generate_embedding(str(row['context']), model_id)
    question_emb = generate_embedding(str(row['question']), model_id)
    
    # For answers, use the first answer text
    answer_text = ""
    if isinstance(row['answers'], dict) and 'text' in row['answers']:
        if isinstance(row['answers']['text'], list) and len(row['answers']['text']) > 0:
            answer_text = str(row['answers']['text'][0])
    answer_emb = generate_embedding(answer_text, model_id)
    
    if title_emb and context_emb and question_emb:
        successful_embeddings += 1
    else:
        failed_embeddings += 1
    
    title_embeddings.append(title_emb)
    context_embeddings.append(context_emb)
    question_embeddings.append(question_emb)
    answer_embeddings.append(answer_emb)

# Add embeddings to dataframe
df_sample['title_vector'] = title_embeddings
df_sample['context_vector'] = context_embeddings
df_sample['question_vector'] = question_embeddings
df_sample['answer_vector'] = answer_embeddings

embedding_time = time.time() - embedding_start

print(f"\n{'='*80}")
print(f"✅ Embedding generation complete!")
print(f"   Total time: {embedding_time:.2f}s")
print(f"   Rate: {len(df_sample)/embedding_time:.2f} docs/s")
print(f"   Success: {successful_embeddings} | Failed: {failed_embeddings}")
print(f"   Vector dimension: {len(title_embeddings[0]) if title_embeddings[0] else 'N/A'}")
print(f"{'='*80}")

🧬 Generating vector embeddings for 1,000 documents
   Fields to vectorize: title, context, question, answers.text

🔍 Verifying model deployment...
   Model ID: nD4n_5kBHWwV4qjRGNg4
   Model state: DEPLOYED
✅ Model is deployed and ready!

🔄 Generating embeddings...
   Progress: 0/1000 (0.0%) - 0.0 docs/s
   Progress: 100/1000 (10.0%) - 7.3 docs/s
   Progress: 100/1000 (10.0%) - 7.3 docs/s
   Progress: 200/1000 (20.0%) - 7.3 docs/s
   Progress: 200/1000 (20.0%) - 7.3 docs/s
   Progress: 300/1000 (30.0%) - 7.3 docs/s
   Progress: 300/1000 (30.0%) - 7.3 docs/s
   Progress: 400/1000 (40.0%) - 7.4 docs/s
   Progress: 400/1000 (40.0%) - 7.4 docs/s
   Progress: 500/1000 (50.0%) - 7.5 docs/s
   Progress: 500/1000 (50.0%) - 7.5 docs/s
   Progress: 600/1000 (60.0%) - 7.5 docs/s
   Progress: 600/1000 (60.0%) - 7.5 docs/s
   Progress: 700/1000 (70.0%) - 7.5 docs/s
   Progress: 700/1000 (70.0%) - 7.5 docs/s
   Progress: 800/1000 (80.0%) - 7.6 docs/s
   Progress: 800/1000 (80.0%) - 7.6 docs/s
   Prog

In [56]:
# Create index with k-NN enabled for vector search
knn_index_name = "squad_with_vectors"

# Delete if exists
if os_client.indices.exists(index=knn_index_name):
    os_client.indices.delete(index=knn_index_name)
    print(f"🗑️  Deleted existing index: {knn_index_name}")

# Get vector dimension from first embedding
vector_dim = len(title_embeddings[0]) if title_embeddings[0] else 768

# Create index with k-NN mappings
# Note: Using Lucene engine (nmslib is deprecated)
# Lucene HNSW parameters:
#   - ef_construction: Controls index build quality/speed tradeoff (default: 100, range: 2-∞)
#   - m: Max number of connections per node (default: 16, recommended: 16-32)
knn_settings = {
    "settings": {
        "index": {
            "knn": True,
            "knn.algo_param.ef_search": 100
        }
    },
    "mappings": {
        "properties": {
            "id": {"type": "keyword"},
            "title": {
                "type": "text",
                "analyzer": "standard"
            },
            "context": {
                "type": "text",
                "analyzer": "standard"
            },
            "question": {
                "type": "text",
                "analyzer": "standard"
            },
            "answers": {
                "type": "nested",
                "properties": {
                    "text": {"type": "text"},
                    "answer_start": {"type": "integer"}
                }
            },
            "title_vector": {
                "type": "knn_vector",
                "dimension": vector_dim,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "lucene",
                    "parameters": {
                        "ef_construction": 128,
                        "m": 16
                    }
                }
            },
            "context_vector": {
                "type": "knn_vector",
                "dimension": vector_dim,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "lucene",
                    "parameters": {
                        "ef_construction": 128,
                        "m": 16
                    }
                }
            },
            "question_vector": {
                "type": "knn_vector",
                "dimension": vector_dim,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "lucene",
                    "parameters": {
                        "ef_construction": 128,
                        "m": 16
                    }
                }
            },
            "answer_vector": {
                "type": "knn_vector",
                "dimension": vector_dim,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "lucene",
                    "parameters": {
                        "ef_construction": 128,
                        "m": 16
                    }
                }
            }
        }
    }
}

os_client.indices.create(index=knn_index_name, body=knn_settings)
print(f"✨ Created k-NN index: {knn_index_name}")
print(f"   Vector dimension: {vector_dim}")

# Prepare bulk data with vectors
def prepare_bulk_data_with_vectors(df, index_name):
    """Prepare data with vector embeddings for bulk ingestion"""
    for _, row in df.iterrows():
        # Check if vectors are valid (not None and not empty)
        title_vec = row['title_vector']
        context_vec = row['context_vector']
        question_vec = row['question_vector']
        answer_vec = row['answer_vector']
        
        # Skip if any required vector is None or empty
        if title_vec is None or context_vec is None or question_vec is None:
            continue
        
        # Additional check for list type and non-empty
        if not isinstance(title_vec, list) or not isinstance(context_vec, list) or not isinstance(question_vec, list):
            continue
            
        if len(title_vec) == 0 or len(context_vec) == 0 or len(question_vec) == 0:
            continue
        
        # Build the source document
        source_doc = {
            "id": row['id'],
            "title": row['title'],
            "context": row['context'],
            "question": row['question'],
            "answers": row['answers'],
            "title_vector": title_vec,
            "context_vector": context_vec,
            "question_vector": question_vec
        }
        
        # Only include answer_vector if it's valid and non-empty
        # OpenSearch will reject knn_vector fields with empty arrays or null values
        if answer_vec and isinstance(answer_vec, list) and len(answer_vec) > 0:
            source_doc["answer_vector"] = answer_vec
            
        doc = {
            "_index": index_name,
            "_id": row['id'],
            "_source": source_doc
        }
        yield doc

# Ingest data with vectors
print(f"\n🚀 Ingesting {len(df_sample):,} documents with vector embeddings...")

ingest_start = time.time()

success_count = 0
error_count = 0

# Use helpers.bulk for efficient ingestion
result_gen = helpers.bulk(
    os_client,
    prepare_bulk_data_with_vectors(df_sample, knn_index_name),
    chunk_size=100,  # Smaller chunk size for vector data
    request_timeout=120,
    raise_on_error=False,
    stats_only=True  # Use stats_only for cleaner output
)

# Result is a tuple: (success_count, errors_list or error_count)
success_count, errors = result_gen
if isinstance(errors, list):
    error_count = len(errors)
    # Print first few errors if any
    for i, error in enumerate(errors[:5], 1):
        print(f"   ⚠️  Error {i}: {error}")
else:
    error_count = errors

ingest_time = time.time() - ingest_start

# Refresh index
os_client.indices.refresh(index=knn_index_name)

# Get stats
stats = os_client.indices.stats(index=knn_index_name)
index_size_bytes = stats['indices'][knn_index_name]['total']['store']['size_in_bytes']
index_size_mb = index_size_bytes / (1024 * 1024)
doc_count = stats['indices'][knn_index_name]['total']['docs']['count']

print(f"\n{'='*80}")
print(f"✅ Vector ingestion complete!")
print(f"   Time: {ingest_time:.2f}s")
print(f"   Rate: {success_count/ingest_time:.2f} docs/s")
print(f"   Success: {success_count} | Errors: {error_count}")
print(f"   Index size: {index_size_mb:.2f} MB")
print(f"   Documents in index: {doc_count:,}")

if error_count > 0:
    print(f"\n⚠️  {error_count} errors occurred during ingestion")

print(f"{'='*80}")

🗑️  Deleted existing index: squad_with_vectors
✨ Created k-NN index: squad_with_vectors
   Vector dimension: 768

🚀 Ingesting 1,000 documents with vector embeddings...

✅ Vector ingestion complete!
   Time: 1.21s
   Rate: 829.36 docs/s
   Success: 1000 | Errors: 0
   Index size: 9.53 MB
   Documents in index: 2,000

✅ Vector ingestion complete!
   Time: 1.21s
   Rate: 829.36 docs/s
   Success: 1000 | Errors: 0
   Index size: 9.53 MB
   Documents in index: 2,000


# ⚡ Phase 4: BM25 vs Neural vs Hybrid Search Comparison

Compare traditional BM25, neural k-NN search, and hybrid approaches.

In [57]:
# Define test queries for comparison
test_search_queries = [
    {
        "name": "Factual Question",
        "text": "What is machine learning?",
        "field": "question"
    },
    {
        "name": "Semantic Concept",
        "text": "artificial intelligence and deep learning",
        "field": "context"
    },
    {
        "name": "Specific Entity",
        "text": "Super Bowl 50",
        "field": "context"
    },
    {
        "name": "Abstract Query",
        "text": "how does training work in neural networks",
        "field": "question"
    }
]

# Function to perform BM25 search
def bm25_search(os_client, index_name, query_text, field):
    """Traditional BM25 text search"""
    start_time = time.time()
    
    try:
        response = os_client.search(
            index=index_name,
            body={
                "query": {
                    "match": {
                        field: query_text
                    }
                },
                "size": 10
            }
        )
        
        latency = (time.time() - start_time) * 1000
        hits = response['hits']['hits']
        
        return {
            "latency_ms": round(latency, 2),
            "total_hits": response['hits']['total']['value'],
            "returned_docs": len(hits),
            "max_score": round(hits[0]['_score'], 4) if hits else 0,
            "avg_score": round(sum(h['_score'] for h in hits) / len(hits), 4) if hits else 0,
            "top_doc_id": hits[0]['_id'] if hits else None,
            "status": "✅"
        }
    except Exception as e:
        return {
            "latency_ms": round((time.time() - start_time) * 1000, 2),
            "total_hits": 0,
            "returned_docs": 0,
            "max_score": 0,
            "avg_score": 0,
            "top_doc_id": None,
            "status": f"❌ {str(e)[:30]}"
        }

# Function to perform Neural k-NN search
def neural_search(os_client, model_id, index_name, query_text, vector_field):
    """Neural search using k-NN vector similarity"""
    start_time = time.time()
    
    try:
        # Generate query embedding
        query_embedding = generate_embedding(query_text, model_id)
        
        if not query_embedding:
            raise Exception("Failed to generate query embedding")
        
        response = os_client.search(
            index=index_name,
            body={
                "query": {
                    "knn": {
                        vector_field: {
                            "vector": query_embedding,
                            "k": 10
                        }
                    }
                },
                "size": 10
            }
        )
        
        latency = (time.time() - start_time) * 1000
        hits = response['hits']['hits']
        
        return {
            "latency_ms": round(latency, 2),
            "total_hits": response['hits']['total']['value'],
            "returned_docs": len(hits),
            "max_score": round(hits[0]['_score'], 4) if hits else 0,
            "avg_score": round(sum(h['_score'] for h in hits) / len(hits), 4) if hits else 0,
            "top_doc_id": hits[0]['_id'] if hits else None,
            "status": "✅"
        }
    except Exception as e:
        return {
            "latency_ms": round((time.time() - start_time) * 1000, 2),
            "total_hits": 0,
            "returned_docs": 0,
            "max_score": 0,
            "avg_score": 0,
            "top_doc_id": None,
            "status": f"❌ {str(e)[:30]}"
        }

# Function to perform Hybrid search
def hybrid_search(os_client, model_id, index_name, query_text, text_field, vector_field):
    """Hybrid search combining BM25 and neural search"""
    start_time = time.time()
    
    try:
        # Generate query embedding
        query_embedding = generate_embedding(query_text, model_id)
        
        if not query_embedding:
            raise Exception("Failed to generate query embedding")
        
        # Hybrid query with normalization
        response = os_client.search(
            index=index_name,
            body={
                "query": {
                    "hybrid": {
                        "queries": [
                            {
                                "match": {
                                    text_field: query_text
                                }
                            },
                            {
                                "knn": {
                                    vector_field: {
                                        "vector": query_embedding,
                                        "k": 10
                                    }
                                }
                            }
                        ]
                    }
                },
                "size": 10
            }
        )
        
        latency = (time.time() - start_time) * 1000
        hits = response['hits']['hits']
        
        return {
            "latency_ms": round(latency, 2),
            "total_hits": response['hits']['total']['value'],
            "returned_docs": len(hits),
            "max_score": round(hits[0]['_score'], 4) if hits else 0,
            "avg_score": round(sum(h['_score'] for h in hits) / len(hits), 4) if hits else 0,
            "top_doc_id": hits[0]['_id'] if hits else None,
            "status": "✅"
        }
    except Exception as e:
        # Fallback: if hybrid not supported, use manual combination
        latency = (time.time() - start_time) * 1000
        return {
            "latency_ms": round(latency, 2),
            "total_hits": 0,
            "returned_docs": 0,
            "max_score": 0,
            "avg_score": 0,
            "top_doc_id": None,
            "status": f"⚠️ Hybrid not supported"
        }

print("✅ Search functions defined")
print("\n📝 Test queries:")
for i, q in enumerate(test_search_queries, 1):
    print(f"  {i}. {q['name']}: '{q['text']}'")

✅ Search functions defined

📝 Test queries:
  1. Factual Question: 'What is machine learning?'
  2. Semantic Concept: 'artificial intelligence and deep learning'
  3. Specific Entity: 'Super Bowl 50'
  4. Abstract Query: 'how does training work in neural networks'


In [58]:
# Execute all three search strategies for comparison
comparison_results = []

# Use the standard analyzer index for BM25 comparison
bm25_index = "squad_standard_analyzer"

print("="*80)
print("🔍 EXECUTING SEARCH STRATEGY COMPARISONS")
print("="*80)

for query in test_search_queries:
    print(f"\n{'─'*80}")
    print(f"📝 Query: {query['name']}")
    print(f"   Text: '{query['text']}'")
    print(f"{'─'*80}")
    
    # Determine vector field based on text field
    vector_field_map = {
        "question": "question_vector",
        "context": "context_vector",
        "title": "title_vector"
    }
    vector_field = vector_field_map.get(query['field'], "question_vector")
    
    # BM25 Search
    print("  🎯 BM25 Search...", end=" ")
    bm25_result = bm25_search(os_client, bm25_index, query['text'], query['field'])
    print(f"{bm25_result['status']} - {bm25_result['latency_ms']}ms")
    
    # Neural Search
    print("  🧠 Neural Search...", end=" ")
    neural_result = neural_search(os_client, model_id, knn_index_name, query['text'], vector_field)
    print(f"{neural_result['status']} - {neural_result['latency_ms']}ms")
    
    # Hybrid Search
    print("  ⚡ Hybrid Search...", end=" ")
    hybrid_result = hybrid_search(os_client, model_id, knn_index_name, query['text'], query['field'], vector_field)
    print(f"{hybrid_result['status']} - {hybrid_result['latency_ms']}ms")
    
    # Store results
    comparison_results.append({
        "query_name": query['name'],
        "query_text": query['text'],
        "strategy": "BM25",
        **bm25_result
    })
    
    comparison_results.append({
        "query_name": query['name'],
        "query_text": query['text'],
        "strategy": "Neural",
        **neural_result
    })
    
    comparison_results.append({
        "query_name": query['name'],
        "query_text": query['text'],
        "strategy": "Hybrid",
        **hybrid_result
    })

print(f"\n{'='*80}")
print("✅ All search strategy comparisons complete!")
print("="*80)

🔍 EXECUTING SEARCH STRATEGY COMPARISONS

────────────────────────────────────────────────────────────────────────────────
📝 Query: Factual Question
   Text: 'What is machine learning?'
────────────────────────────────────────────────────────────────────────────────
  🎯 BM25 Search... ✅ - 24.85ms
  🧠 Neural Search... ✅ - 80.63ms
  ⚡ Hybrid Search... ✅ - 81.02ms

────────────────────────────────────────────────────────────────────────────────
📝 Query: Semantic Concept
   Text: 'artificial intelligence and deep learning'
────────────────────────────────────────────────────────────────────────────────
  🎯 BM25 Search... ✅ - 23.66ms
  🧠 Neural Search... ✅ - 41.9ms
  ⚡ Hybrid Search... ✅ - 65.7ms

────────────────────────────────────────────────────────────────────────────────
📝 Query: Specific Entity
   Text: 'Super Bowl 50'
────────────────────────────────────────────────────────────────────────────────
  🎯 BM25 Search... ✅ - 9.88ms
  🧠 Neural Search... ✅ - 27.45ms
  ⚡ Hybrid Search... ✅ -

## 📊 Table 3: BM25 vs Neural vs Hybrid Search - Final Comparison

In [59]:
# Create comprehensive final comparison table
df_comparison = pd.DataFrame(comparison_results)

print("="*120)
print("📊 DETAILED COMPARISON BY QUERY")
print("="*120)

for query_name in df_comparison['query_name'].unique():
    query_data = df_comparison[df_comparison['query_name'] == query_name]
    
    print(f"\n🔍 {query_name}")
    print(f"   Query: '{query_data.iloc[0]['query_text']}'")
    print("-" * 120)
    
    # Create display dataframe
    display_df = query_data[['strategy', 'latency_ms', 'total_hits', 'returned_docs', 'max_score', 'avg_score', 'status']].copy()
    display_df.columns = ['Strategy', 'Latency (ms)', 'Total Hits', 'Docs', 'Max Score', 'Avg Score', 'Status']
    
    print(display_df.to_string(index=False))
    
    # Highlight winner
    valid_results = display_df[display_df['Status'] == '✅']
    if len(valid_results) > 0:
        fastest = valid_results.loc[valid_results['Latency (ms)'].idxmin()]
        best_score = valid_results.loc[valid_results['Max Score'].idxmax()]
        print(f"\n   ⚡ Fastest: {fastest['Strategy']} ({fastest['Latency (ms)']}ms)")
        print(f"   ⭐ Best Score: {best_score['Strategy']} ({best_score['Max Score']:.4f})")
    print()

# Summary statistics by strategy
print("\n" + "="*120)
print("📈 OVERALL STRATEGY PERFORMANCE")
print("="*120)

strategy_summary = df_comparison.groupby('strategy').agg({
    'latency_ms': ['mean', 'min', 'max'],
    'total_hits': 'mean',
    'max_score': 'mean',
    'avg_score': 'mean',
    'returned_docs': 'mean'
}).round(2)

strategy_summary.columns = ['Avg Latency', 'Min Latency', 'Max Latency', 'Avg Total Hits', 'Avg Max Score', 'Avg Score', 'Avg Docs']
strategy_summary = strategy_summary.sort_values('Avg Latency')

print(strategy_summary.to_string())

# Create styled comparison table with recommendations
recommendations = {
    "BM25": {
        "strength": "Fast, keyword-focused, exact term matching",
        "weakness": "May miss semantic meaning, synonym issues",
        "use_case": "Exact match, known entities, structured queries"
    },
    "Neural": {
        "strength": "Semantic understanding, handles synonyms/paraphrasing",
        "weakness": "Slower due to embedding generation, resource intensive",
        "use_case": "Conceptual search, natural language queries, cross-lingual"
    },
    "Hybrid": {
        "strength": "Best of both worlds, balanced precision/recall",
        "weakness": "Most complex, may need tuning, highest latency",
        "use_case": "Production systems needing both precision and semantic understanding"
    }
}

# Create detailed comparison HTML table
html_comparison = """
<style>
    .final-comparison {{
        width: 100%;
        border-collapse: collapse;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        box-shadow: 0 4px 12px rgba(0,0,0,0.15);
        margin: 20px 0;
    }}
    .final-comparison th {{
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 15px;
        text-align: left;
        font-weight: 600;
        font-size: 14px;
        border: 1px solid #5568d3;
    }}
    .final-comparison td {{
        padding: 12px 15px;
        border: 1px solid #e0e0e0;
        font-size: 13px;
    }}
    .final-comparison tr:hover {{
        background-color: #f5f7ff;
    }}
    .final-comparison tr:nth-child(even) {{
        background-color: #fafbff;
    }}
    .strategy-bm25 {{ background-color: #e3f2fd !important; }}
    .strategy-neural {{ background-color: #f3e5f5 !important; }}
    .strategy-hybrid {{ background-color: #fff3e0 !important; }}
    .recommendation {{
        margin-top: 20px;
        padding: 15px;
        border-left: 4px solid #667eea;
        background-color: #f5f7ff;
        border-radius: 4px;
    }}
</style>

<div class="final-comparison">
    <h2>🏆 Search Strategy Comparison & Recommendations</h2>
    <table class="final-comparison">
        <thead>
            <tr>
                <th>Strategy</th>
                <th>Avg Latency (ms)</th>
                <th>Avg Max Score</th>
                <th>💪 Strengths</th>
                <th>⚠️ Weaknesses</th>
                <th>✅ Best Use Cases</th>
            </tr>
        </thead>
        <tbody>
"""

for strategy in ['BM25', 'Neural', 'Hybrid']:
    if strategy in strategy_summary.index:
        stats = strategy_summary.loc[strategy]
        rec = recommendations[strategy]
        row_class = f"strategy-{strategy.lower()}"
        
        html_comparison += f"""
            <tr class="{row_class}">
                <td><strong>{strategy}</strong></td>
                <td>{stats['Avg Latency']:.2f} ms</td>
                <td>{stats['Avg Max Score']:.4f}</td>
                <td>{rec['strength']}</td>
                <td>{rec['weakness']}</td>
                <td>{rec['use_case']}</td>
            </tr>
        """

html_comparison += """
        </tbody>
    </table>
</div>

<div class="recommendation">
    <h3>💡 Key Insights & Recommendations</h3>
    <ul>
        <li><strong>Speed Priority:</strong> Use <strong>BM25</strong> for fastest response times and exact keyword matching</li>
        <li><strong>Semantic Understanding:</strong> Use <strong>Neural Search</strong> when meaning matters more than exact terms</li>
        <li><strong>Production Systems:</strong> Use <strong>Hybrid Search</strong> for best overall results combining precision and recall</li>
        <li><strong>Cost Consideration:</strong> Neural search requires more compute resources for embedding generation</li>
        <li><strong>Index Size:</strong> Vector embeddings significantly increase index size (see Phase 1 comparison)</li>
    </ul>
</div>
"""

display(HTML(html_comparison))

# Print final summary
print("\n" + "="*120)
print("🎯 FINAL RECOMMENDATIONS")
print("="*120)
print("\n📊 Performance Summary:")
fastest_strategy = strategy_summary.index[0]
best_score_strategy = strategy_summary['Avg Max Score'].idxmax()

print(f"   ⚡ Fastest: {fastest_strategy} ({strategy_summary.loc[fastest_strategy, 'Avg Latency']:.2f}ms average)")
print(f"   ⭐ Best Relevance: {best_score_strategy} ({strategy_summary.loc[best_score_strategy, 'Avg Max Score']:.4f} average score)")

print("\n💡 Choose Based On:")
print("   • BM25: Simple keyword searches, known entities, minimal resources")
print("   • Neural: Semantic/conceptual queries, multilingual, paraphrase handling")  
print("   • Hybrid: Production apps needing both precision and semantic understanding")

print("\n⚠️  Important Notes:")
print("   • Neural search requires deployed ML model (memory intensive)")
print("   • Vector indices are larger (embeddings for each field)")
print("   • Hybrid may not be available in all OpenSearch versions")
print("   • Consider query patterns and user needs when choosing strategy")

print("\n" + "="*120)

📊 DETAILED COMPARISON BY QUERY

🔍 Factual Question
   Query: 'What is machine learning?'
------------------------------------------------------------------------------------------------------------------------
Strategy  Latency (ms)  Total Hits  Docs     Max Score     Avg Score Status
    BM25         24.85       10000    10  8.711500e+00  4.927000e+00      ✅
  Neural         80.63          10    10  8.630000e-01  8.580000e-01      ✅
  Hybrid         81.02         463    10 -9.549512e+09 -1.397195e+09      ✅

   ⚡ Fastest: BM25 (24.85ms)
   ⭐ Best Score: BM25 (8.7115)


🔍 Semantic Concept
   Query: 'artificial intelligence and deep learning'
------------------------------------------------------------------------------------------------------------------------
Strategy  Latency (ms)  Total Hits  Docs     Max Score     Avg Score Status
    BM25         23.66       10000    10  9.550200e+00  6.513300e+00      ✅
  Neural         41.90          10    10  8.428000e-01  8.428000e-01      ✅
 

Strategy,Avg Latency (ms),Avg Max Score,💪 Strengths,⚠️ Weaknesses,✅ Best Use Cases
BM25,18.78 ms,8.4000,"Fast, keyword-focused, exact term matching","May miss semantic meaning, synonym issues","Exact match, known entities, structured queries"
Neural,46.75 ms,0.8600,"Semantic understanding, handles synonyms/paraphrasing","Slower due to embedding generation, resource intensive","Conceptual search, natural language queries, cross-lingual"
Hybrid,57.84 ms,-9549512000.0000,"Best of both worlds, balanced precision/recall","Most complex, may need tuning, highest latency",Production systems needing both precision and semantic understanding



🎯 FINAL RECOMMENDATIONS

📊 Performance Summary:
   ⚡ Fastest: BM25 (18.78ms average)
   ⭐ Best Relevance: BM25 (8.4000 average score)

💡 Choose Based On:
   • BM25: Simple keyword searches, known entities, minimal resources
   • Neural: Semantic/conceptual queries, multilingual, paraphrase handling
   • Hybrid: Production apps needing both precision and semantic understanding

⚠️  Important Notes:
   • Neural search requires deployed ML model (memory intensive)
   • Vector indices are larger (embeddings for each field)
   • Hybrid may not be available in all OpenSearch versions
   • Consider query patterns and user needs when choosing strategy



## 🎓 Summary & Conclusion

This comprehensive analysis compared multiple approaches for indexing and searching the SQUAD dataset in OpenSearch:

### 📊 Phase 1: Text Analyzers
We tested 5 different analyzers (standard, english, keyword, whitespace, simple) and measured:
- **Ingestion Performance**: Throughput (docs/sec), time, and index size
- **Key Finding**: Different analyzers have similar ingestion speeds but vary in index size

### 🔍 Phase 2: Query Performance  
We executed various query types (match, multi_match, match_phrase) across all analyzer indices:
- **Latency Comparison**: Measured query response times
- **Relevance Scoring**: Compared how well each analyzer returned relevant results
- **Key Finding**: Analyzer choice significantly impacts relevance for different query types

### 🧠 Phase 3: Neural Search
We deployed a sentence transformer model and created vector embeddings:
- **Model**: huggingface/sentence-transformers/msmarco-distilbert-base-tas-b
- **Vectorized Fields**: title, context, question, answers (all except id)
- **Trade-offs**: Slower ingestion, larger index size, but semantic understanding

### ⚡ Phase 4: Search Strategy Comparison
We compared three search approaches:
1. **BM25 (Traditional)**: Fast lexical matching
2. **Neural (k-NN)**: Semantic vector similarity  
3. **Hybrid**: Combined approach

### 🎯 Key Takeaways
- **Speed vs Semantics**: BM25 is fastest but neural search understands meaning
- **Resource Requirements**: Neural search needs deployed models and more storage
- **Production Recommendation**: Hybrid search for best overall results
- **Use Case Matters**: Choose strategy based on query patterns and requirements

---

**Next Steps**: 
- Tune analyzer settings for specific use cases
- Experiment with different embedding models
- Implement re-ranking strategies
- Add search result evaluation metrics (NDCG, MRR, MAP)